## Importing the Required Libraries

In [4]:
import nltk
#nltk.download("stopwords")
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis visualization 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# import warnings
# warnings.filterwarnings("ignore", category=DepreciationWarning)

c:\users\kenny\envs\interestinvest\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Preparing Data

In [5]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [6]:
stopwords = stopwords.words("english")

In [8]:
data = load_data("data/testData.json")
print(list(data.values()))

['D. R. Horton, Inc. engages in the construction and sale of single-family housing. It operates through the following segments: Homebuilding and Financial Services. The Homebuilding segment includes the sub-segments East, Midwest, Southeast, South Central, Southwest and West regions. The Financial Services segment provides mortgage financing and title agency services to homebuyers in many of its homebuilding markets. The company was founded by Donald Ray Horton in 1978 and is headquartered in Arlington, TX.']


In [10]:
def lemmatization(texts, allowed_postags=["NOUN"]): # only allow nouns 
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
                
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


In [13]:
lemmatized_text = lemmatization(list(data.values()))
print(lemmatized_text)

['construction sale family housing segment homebuilding segment sub - segment region segment mortgage financing title agency service homebuyer homebuilding market company']


In [100]:
def gen_words(text):
    new = gensim.utils.simple_preprocess(text, deacc=True)
    return [new]

data_words = gen_words(lemmatized_text)
print(data_words)

[['construction', 'sale', 'family', 'housing', 'segment', 'homebuilding', 'segment', 'sub', 'segment', 'region', 'segment', 'mortgage', 'financing', 'title', 'agency', 'service', 'homebuyer', 'homebuilding', 'market', 'company']]


In [102]:
id2word = corpora.Dictionary(data_words)
print(id2word)
corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
print (corpus)


Dictionary(16 unique tokens: ['agency', 'company', 'construction', 'family', 'financing']...)
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 4), (13, 1), (14, 1), (15, 1)]]


In [103]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [105]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

c:\users\kenny\envs\interestinvest\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.019800 -0.013649       1        1  97.131258
0      0.000683  0.000471       2        1   0.098922
16     0.000683  0.000471       3        1   0.098922
28     0.000683  0.000471       4        1   0.098922
27     0.000683  0.000471       5        1   0.098922
26     0.000683  0.000471       6        1   0.098922
25     0.000683  0.000471       7        1   0.098922
24     0.000683  0.000471       8        1   0.098922
23     0.000683  0.000471       9        1   0.098922
22     0.000683  0.000471      10        1   0.098922
21     0.000683  0.000471      11        1   0.098922
20     0.000683  0.000471      12        1   0.098922
19     0.000683  0.000471      13        1   0.098922
18     0.000683  0.000471      14        1   0.098922
17     0.000683  0.000471      15        1   0.098922
15     0.000683  0.000471      16        1   0.098922
1      0.000683  0.000471      17        1   0.098922
14     0.000683  0.000471      18        1   0.098922
13     0.000683  0.000471      19        1   0.098922
12     0.000683  0.000471      20        1   0.098922
10     0.000683  0.000471      21        1   0.098922
9      0.000683  0.000471      22        1   0.098922
8      0.000683  0.000471      23        1   0.098922
7      0.000683  0.000471      24        1   0.098922
6      0.000683  0.000471      25        1   0.098922
5      0.000683  0.000471      26        1   0.098922
4      0.000683  0.000471      27        1   0.098922
3      0.000683  0.000471      28        1   0.098922
2      0.000683  0.000471      29        1   0.098922
29     0.000683  0.000471      30        1   0.098922, topic_info=            Term      Freq     Total Category  logprob  loglift
12       segment  3.000000  3.000000  Default  16.0000  16.0000
5   homebuilding  1.000000  1.000000  Default  15.0000  15.0000
0         agency  1.000000  1.000000  Default  14.0000  14.0000
1        company  1.000000  1.000000  Default  13.0000  13.0000
2   construction  1.000000  1.000000  Default  12.0000  12.0000
..           ...       ...       ...      ...      ...      ...
13       service  0.001237  1.013478  Topic30  -2.7726   0.2098
14           sub  0.001237  1.013478  Topic30  -2.7726   0.2098
15         title  0.001237  1.013478  Topic30  -2.7726   0.2098
5   homebuilding  0.001237  1.959566  Topic30  -2.7726  -0.4496
12       segment  0.001237  3.851740  Topic30  -2.7726  -1.1254

[496 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         1  0.986701        agency
1         1  0.986701       company
2         1  0.986701  construction
3         1  0.986701        family
4         1  0.986701     financing
5         1  1.020634  homebuilding
6         1  0.986701     homebuyer
7         1  0.986701       housing
8         1  0.986701        market
9         1  0.986701      mortgage
10        1  0.986701        region
11        1  0.986701          sale
12        1  1.038492       segment
13        1  0.986701       service
14        1  0.986701           sub
15        1  0.986701         title, R=16, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 1, 17, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 16, 2, 15, 14, 13, 11, 10, 9, 8, 7, 6, 5, 4, 3, 30])